<a href="https://colab.research.google.com/github/Muhammad-Saad-2/LangLearn/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing encessary libraries

%pip install langchain -q -U
%pip install langchain-community -q -U
%pip install langchain-huggingface -q -U
%pip install langchain_pinecone -q -U
%pip install sentence-transformers -q -U
%pip install langchain-google-genai -q -U
%pip install langchain-pinecone -q -U

In [ ]:
import getpass
import os
import time
from google.colab import userdata
from pinecone import Pinecone

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc =  Pinecone(api_key=pinecone_api_key)

In [ ]:
#creating an index in pinecone in order to store the data in an organized way

import time

index_name = "ragexplore"

if index_name :
  print("index already exists")
else:
  pc.create_index(
      name=index_name,
      dimension=384,
      metric="cosine",
      spec=ServerlessSpec(cloud="aws", region="us-east-1"),

)
index = pc.Index(index_name)

index already exists


In [ ]:
#text embedding instance

from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore

embedding = SentenceTransformer( "sentence-transformers/all-mpnet-base-v2")

def embed_function(texts):
  if isinstance(texts, str):
    return embedding.encode(texts)
  return [embedding.encode(text) for text in texts]


vector_store = PineconeVectorStore(
    index=index,
    embedding=embedding,
    text_key="text"

  )


In [ ]:
from langchain_core.documents import Document
from uuid import uuid4
from sentence_transformers import SentenceTransformer



document_1 = Document(
    page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
    metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
)
document_2 = Document(
    page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
    metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
)
document_3 = Document(
    page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
    metadata={"year": 200, "director": "Satoshi Kon", "rating": 8.6},
)
document_4 = Document(
    page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
    metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
)
document_5 = Document(
    page_content="Toys come alive and have a blast doing so",
    metadata={"year": 1995, "genre": "animated"},
)
document_6 = Document(
    page_content="Three men walk into the Zone, three men walk out of the Zone",
    metadata={"year": 19, "rating": 8.3},
)
document_7 = Document(
    page_content="In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization",
    metadata={"year": 2004, "director": "Steven Spielberg", "rating": 8.6},
)
document_8 = Document(
    page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
    metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
)
document_9 = Document(
    page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
    metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
)
document_10 = Document(
    page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
    metadata={"year": 200, "director": "Satoshi Kon", "rating": 8.6},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10
  ]


uuids = [str(uuid4()) for doc in range (0, len(documents))]





In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(model_name)

embeddings = []

for document in documents:
  embedding = model.encode(document.page_content)
  embeddings.append(embedding)

In [ ]:
# prompt: now pass the embeded documents into gemini model using the API keys in my secrets and generate a mixture of my embed documents and ai model's generated text

import os
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI

# Replace with your actual API key
google_api_key = userdata.get('GOOGLE_API_KEY')

# Initialize the Gemini model, using 'gemini-pro' as the model family
# Valid options for model include: 'gemini-pro', 'gemini-turbo', 'text-bison@001'
llm = GoogleGenerativeAI(model="gemini-pro", api_key=google_api_key, temperature=0.7)  # Adjust temperature as needed

# Example usage with the documents and embeddings
generated_text = llm(
    "Generate a creative text that incorporates information from these movie summaries:\n\n"
    + "\n\n".join([doc.page_content for doc in documents])
    + "\n\nThe new text should be a cohesive story, not just a summary."
)
generated_text

<ipython-input-31-47d58b73ff14>:15: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  generated_text = llm(


"In a realm where reality blurred and dreams intertwined, a group of audacious scientists dared to defy nature's boundaries. They ventured into the depths of the past, retrieving remnants of an ancient world: dinosaurs. As these prehistoric behemoths were reanimated, chaos erupted, their thunderous footsteps shaking the very foundations of civilization.\n\nAmidst this pandemonium, a brilliant psychologist embarked on a perilous journey into the labyrinthine realm of dreams. Each layer of slumber revealed a deeper abyss, where memories, desires, and fears danced in an enigmatic ballet. As he delved further into this surreal landscape, he encountered a young man who had become trapped in a perpetual cycle of dreams within dreams, a haunting mirror image of his own predicament.\n\nMeanwhile, in a quaint suburban town, a group of virtuous women exuded an otherworldly wholesomeness. Their simple lives stood in stark contrast to the turmoil raging outside their cozy homes. Men from far and w

In [ ]:
dir(PineconeVectorStore)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_cosine_relevance_score_fn',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_texts',
 'adelete',
 'afrom_documents',
 'afrom_texts',
 'aget_by_ids',
 'amax_marginal_relevance_search',
 'amax_marginal_relevance_search_by_vector',
 'as_retriever',
 'asearch',
 'asimilarity_search',
 'asimilarity_search_by_ve

In [ ]:
dir(SentenceTransformer)

['T_destination',
 '__add__',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_compiled_call_impl',
 '_create_model_card',
 '_encode_multi_process_worker',
 '_eval_during_training',
 '_first_module',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_item_by_idx',
 '_get_name',
 '_get_scheduler',
 '_keys_to_ignore_on_save',
 '_last_module',
 '_load_auto_model',
 '_load_from_state_dict',
 '_load_module_class_from_ref',
 '_load_sbert